Copyright (c) 2023 Habana Labs, Ltd. an Intel Company.
#### Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License. You may obtain a copy of the License at https://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
# Intel&reg; Gaudi&reg; 2 Model Profiling and Optimization using HuggingFace

## Objective
This tutorial will show the user how to run the Intel Gaudi Profiling tools: the habana_perf_tool and the Tensorboard plug-in on the Intel Gaudi 2 AI Accelerator, and the profiling trace viewer.  These tools will provide the user valueable optimization tips and information to modify any model for better performance.   Following these steps and using these tools can help you better understand some of the bottlenecks of your model.  For more information, please refer to the [Profiling](https://docs.habana.ai/en/latest/Profiling/index.html) section of the documentation for info on how to setup the profiler and the [Optimization Guide](https://docs.habana.ai/en/latest/PyTorch/Model_Optimization_PyTorch/index.html) for additional background on other optimization techniques.

| Task                                 | Description                                             | Details                                         |
|--------------------------------------|---------------------------------------------------------|-------------------------------------------------|
| PyTorch Profiling with TensorBoard   | Obtains Gaudi-specific recommendations for performance using TensorBoard. | [Profiling with PyTorch](https://docs.habana.ai/en/latest/Profiling/Profiling_with_PyTorch.html#profiling-with-pytorch)        |
| Review the PT_HPU_METRICS_FILE      | Looks for excessive re-compilations during runtime.     | [Runtime Environment Variables](https://docs.habana.ai/en/latest/PyTorch/Reference/Runtime_Flags.html#pytorch-runtime-flags)                   |                         
| Profiling Trace Viewer               | Uses Perfetto to view traces.           |  [Getting Started with Intel Gaudi Profiler](https://docs.habana.ai/en/latest/Profiling/Intel_Gaudi_Profiling/Getting_Started_with_Profiler.html#getting-started-with-profiler)                      |                         
| Model Logging                        | Sets ENABLE_CONSOLE to set Logging for debug and analysis. | [Runtime Environment Variables](https://docs.habana.ai/en/latest/PyTorch/Reference/Runtime_Flags.html#pytorch-runtime-flags)                |                         




### Initial Setup
To run the this jupyter notebook and the Tensorboard viewer, set the appropriate ports for access when you ssh into the Intel Gaudi 2 node. you need to ensure that the following ports are open:
* 8888 (for running this jupyter notebook)
* 6006 (for running Tensorboard)    

Do to this, you need to add the following in your overall ssh commmand when connecting to the Intel Gaudi Node:

`ssh -L 8888:localhost:8888 -L 6006:localhost:6006 .... `

We start with an Intel Gaudi PyTorch Docker image and run this notebook.   For this example, we'll be using the [Swin Transformer](https://huggingface.co/microsoft/swin-base-patch4-window7-224-in22k) model from the Hugging Face Repository running on Hugging Face's Optimum-Habana library.  So the first step is to load the Optimum-Habana library and model repository:

In [2]:
%cd ~/ctao/Gaudi-tutorials/PyTorch/Profiling_and_Optimization
!pip install pickleshare ipython
!pip install optimum-habana==1.13.2

/home/demo/ctao/Gaudi-tutorials/PyTorch/Profiling_and_Optimization


/home/demo/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/home/demo/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
  Using cached optimum_habana-1.13.2-py3-none-any.whl (565 kB)
  Using cached sentence_transformers-3.0.1-py3-none-any.whl (227 kB)
  Using cached optimum-1.22.0-py3-none-any.whl (453 kB)
  Using cached diffusers-0.29.2-py3-none-any.whl (2.2 MB)
  Using cached transformers-4.43.4-py3-none-any.whl (9.4 MB)
  Using cached accelerate-0.33.0-py3-none-any.whl (315 kB)
  Using cached numpy-2.1.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.3 MB)
  Using cached scipy-1.14.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (41.2 MB)
  Using cached scikit_learn-1.5.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.3 MB)
  Using cached datasets-3.0.1-py3-none-any.whl (471 kB)
  Using cached num

In [3]:
!git clone https://github.com/huggingface/optimum-habana
%cd optimum-habana
!git checkout v1.13.2
%cd ..

Cloning into 'optimum-habana'...


remote: Enumerating objects: 18189, done.
remote: Counting objects: 100% (1770/1770), done.
remote: Compressing objects: 100% (841/841), done.
remote: Total 18189 (delta 1181), reused 1293 (delta 820), pack-reused 16419 (from 1)
Receiving objects: 100% (18189/18189), 11.85 MiB | 23.70 MiB/s, done.
Resolving deltas: 100% (12520/12520), done.
/home/demo/ctao/Gaudi-tutorials/PyTorch/Profiling_and_Optimization/optimum-habana
Note: switching to 'v1.13.2'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at 1266993d Rele

We now will go into the image-classification task and load the specfic requirements for the task:

In [4]:
%cd optimum-habana/examples/image-classification
!pip install -r requirements.txt

/home/demo/ctao/Gaudi-tutorials/PyTorch/Profiling_and_Optimization/optimum-habana/examples/image-classification


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 17.9 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


### Running the Model
Now that the model is loaded, we'll run the model and look for the trace files for analysis. 

For this model script we can see the profiling set in the utils.py. 
For other models not in optimum-habana, users can refer to [Profiling_with_PyTorch](https://docs.habana.ai/en/latest/Profiling/Profiling_with_PyTorch.html) to setup profiling 

In [5]:
%%sh

cat -n ../../optimum/habana/utils.py | head -n 313 | tail -n 13

   301	            schedule = torch.profiler.schedule(wait=wait, warmup=warmup, active=active, repeat=1)
   302	            activities = [torch.profiler.ProfilerActivity.CPU, torch.profiler.ProfilerActivity.HPU]
   303	
   304	            profiler = torch.profiler.profile(
   305	                schedule=schedule,
   306	                activities=activities,
   307	                on_trace_ready=torch.profiler.tensorboard_trace_handler(output_dir),
   308	                record_shapes=record_shapes,
   309	                with_stack=False,
   310	            )
   311	            self.start = profiler.start
   312	            self.stop = profiler.stop
   313	            self.step = profiler.step


Run Model to collect trace file (unoptimized)
Swin Transformer is a model that capably serves as a general-purpose backbone for computer vision. run_image_classification.py is a script that showcases how to fine-tune Swin Transformer on HPUs.

Notice the torch profiler specific commands:

- `--profiling_warmup_steps 10` - profiler will wait for warmup steps
- `--profiling_steps 3` - records for the next active steps  
                             
The collected trace files will be saved to ./hpu_profile

In [13]:
!python3 run_image_classification.py \
    --model_name_or_path microsoft/swin-base-patch4-window7-224-in22k \
    --dataset_name cifar10 \
    --output_dir /tmp/outputs/ \
    --remove_unused_columns False \
    --image_column_name img \
    --do_train \
    --learning_rate 3e-5 \
    --num_train_epochs 2 \
    --per_device_train_batch_size 64 \
    --evaluation_strategy no \
    --save_strategy no \
    --load_best_model_at_end False \
    --save_total_limit 3 \
    --seed 1337 \
    --use_habana \
    --use_lazy_mode \
    --use_hpu_graphs_for_training \
    --gaudi_config_name Habana/swin \
    --throughput_warmup_steps 3 \
    --bf16 \
    --report_to none \
    --throughput_warmup_steps 2 \
    --overwrite_output_dir \
    --ignore_mismatched_sizes \
    --profiling_warmup_steps 10 \
    --profiling_steps 3

[WARNING|utils.py:212] 2024-09-28 00:15:35,519 >> optimum-habana v1.13.2 has been validated for SynapseAI v1.17.0 but habana-frameworks v1.16.2.2 was found, this could lead to undefined behavior!
[WARNING|utils.py:225] 2024-09-28 00:15:36,431 >> optimum-habana v1.13.2 has been validated for SynapseAI v1.17.0 but the driver version is v1.16.2, this could lead to undefined behavior!
/home/demo/.local/lib/python3.10/site-packages/optimum/habana/transformers/training_args.py:366: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/demo/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
gaudi_config.json: 100%|██████████████████████| 90.0/90.0 [00:00<00:00, 315kB

In [14]:
%cd hpu_profile
%ls -al

/home/demo/ctao/Gaudi-tutorials/PyTorch/Profiling_and_Optimization/optimum-habana/examples/image-classification/hpu_profile
total 183168
drwxrwxr-x 2 demo demo      4096 Sep 28 00:16 ./
drwxrwxr-x 4 demo demo      4096 Sep 28 00:16 ../
-rw-rw-r-- 1 demo demo 187553452 Sep 28 00:16 hls2-srv02-demolab_2152374.1727482606813402488.pt.trace.json


/home/demo/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


### Reviewing the Details in Tensorboard and perf_tool
Now that the training is completed, you can see the trace files (...pt.trace.json) have been generated and now can be viewed.  Two types of information are produced by TensorBoard:

Model Performance Tracking - While your workload is being processed in batches, you can track the progress of the training process on the dashboard in real-time by monitoring the model’s cost (loss) and accuracy.

Profiling Analysis - Right after the last requested step was completed, the collected profiling data is analyzed by TensorBoard and then immediately submitted to your browser, without any need to wait till the training process is completed.

In [3]:
%reload_ext tensorboard
%tensorboard --logdir=/home/demo/ctao/Gaudi-tutorials/PyTorch/Profiling_and_Optimization/optimum-habana/examples/image-classification/hpu_profile --port 6006    # Your port selection may vary, default is 6006

If you do not want to run the TensorBoard UI, you can take the same .json log files and use the habana_perf_tool that will parse the existing .json file and provide the same recommendations for performance enhancements, but in a text form.

In [19]:
!habana_perf_tool --trace /home/demo/ctao/Gaudi-tutorials/PyTorch/Profiling_and_Optimization/optimum-habana/examples/image-classification/hpu_profile/hls2-srv02-demolab_2152374.1727482606813402488.pt.trace.json

2024-09-28 00:24:19,779 - pytorch_profiler - DEBUG - Loading /home/demo/ctao/Gaudi-tutorials/PyTorch/Profiling_and_Optimization/optimum-habana/examples/image-classification/hpu_profile/hls2-srv02-demolab_2152374.1727482606813402488.pt.trace.json
Import Data (KB): 100%|█████████████| 183157/183157 [00:01<00:00, 131841.86it/s]
2024-09-28 00:24:22,182 - pytorch_profiler - DEBUG - Please wait for initialization to finish ...
2024-09-28 00:24:28,206 - pytorch_profiler - DEBUG - PT Track ids: BridgeTrackIds.Result(pt_bridge_launch='48,34,31', pt_bridge_compute='32', pt_mem_copy='34', pt_mem_log='', pt_build_graph='33,50,51,47')
2024-09-28 00:24:28,207 - pytorch_profiler - DEBUG - Track ids: TrackIds.Result(forward='30', backward='46', synapse_launch='0,2,49', synapse_wait='1,36', device_mme='42,43,44,45', device_tpc='6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29', device_dma='5,37,38,39,40,41')
2024-09-28 00:24:29,021 - pytorch_profiler - DEBUG - Device ratio: 34.41 % (

### Using the Perfetto Trace Viewer
Finally, to view the details of the Intel Gaudi Device itself, you can view the traces in the perfetto trace viewer.  

This step requires you to set the `hl-prof-config` settings and the Environment variable `HABANA_PROFILE=1` as shown below, this will generate the .hltv file that can be viewed using https://perfetto.habana.ai.  Since this is using the Gaudi profiler, the runtime profiling commands need to be removed.  At the end of this run, you will see a `my_profiling_session_12345.hltv` file that can be loaded into the Perfetto browser.

For More Information to enable your model to use the Habana Perfetto Trace viewer, you can refer to the documentation https://docs.habana.ai/en/latest/Profiling/Intel_Gaudi_Profiling/Getting_Started_with_Profiler.html

In [4]:
%cd ..
!hl-prof-config -e off -phase=multi-enq -g 1-20 -s my_profiling_session
!export HABANA_PROFILE=1

/home/demo/ctao/Gaudi-tutorials/PyTorch
Resetting config file /home/demo/.habana/prof_config.json
Edited file /home/demo/.habana/prof_config.json


In [7]:
!HABANA_PROFILE=1 python3 /home/demo/ctao/Gaudi-tutorials/PyTorch/Profiling_and_Optimization/optimum-habana/examples/image-classification/run_image_classification.py \
    --model_name_or_path microsoft/swin-base-patch4-window7-224-in22k \
    --dataset_name cifar10 \
    --output_dir /tmp/outputs/ \
    --remove_unused_columns False \
    --image_column_name img \
    --do_train \
    --learning_rate 3e-5 \
    --num_train_epochs 2 \
    --per_device_train_batch_size 64 \
    --evaluation_strategy no \
    --save_strategy no \
    --load_best_model_at_end False \
    --save_total_limit 3 \
    --seed 1337 \
    --use_habana \
    --use_lazy_mode \
    --report_to none \
    --use_hpu_graphs_for_training \
    --gaudi_config_name Habana/swin \
    --throughput_warmup_steps 3 \
    --bf16 \
    --throughput_warmup_steps 2 \
    --overwrite_output_dir \
    --ignore_mismatched_sizes 
    #--profiling_warmup_steps 10 \
    #--profiling_steps 3

[WARNING|utils.py:212] 2024-09-29 00:13:49,095 >> optimum-habana v1.13.2 has been validated for SynapseAI v1.17.0 but habana-frameworks v1.16.2.2 was found, this could lead to undefined behavior!
[WARNING|utils.py:225] 2024-09-29 00:13:49,983 >> optimum-habana v1.13.2 has been validated for SynapseAI v1.17.0 but the driver version is v1.16.2, this could lead to undefined behavior!
/home/demo/.local/lib/python3.10/site-packages/optimum/habana/transformers/training_args.py:366: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/demo/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/demo/.local/lib/python3.10/site-packages/transformers/deepspeed.py:24: 